# Setup

In [2]:
import numpy as np
from scipy.special import comb
from scipy.optimize import linear_sum_assignment

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
drive_path='drive/My Drive/Colab Notebooks/data_IISc experiment2/'

# Bethe Permanent

In [5]:
def permanent(A):
    assert(len(A.shape)==2),"Input should be a 2D matrix"
    assert(A.shape[0]==A.shape[1]),"Input should be a square matrix"
    
    perm=np.float128(0)
    n=A.shape[1]
    C=2**n
    
    for k in range(1,C):
        rowsumprod=np.float128(1)
        chi=np.array(list(np.binary_repr(k))).astype(np.float128)
        for m in range(0,n):
            rowsum=np.float128(0)
            
            for p in range(0,len(chi)):
                rowsum += chi[p]*A[m,len(chi)-p-1]
            
            rowsumprod *= rowsum
            
        perm += ((-1)**(n-chi.sum()))*rowsumprod
    
    return perm

In [6]:
def belief_prop(P,max_iter=50, eps=0.1):
    n=P.shape[0]
    phi=np.sqrt(P)
    m_x2y=np.ones((n,n,n),dtype=np.float128) 
    m_y2x=np.ones((n,n,n),dtype=np.float128) 
    m_x2y_new=np.ones((n,n,n),dtype=np.float128) 
    m_y2x_new=np.ones((n,n,n),dtype=np.float128) 
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)#belief_xy(i,j,xi,yj) is the value of b(x_i,y_j) for a given value of x_i and y_j
    belief_x=np.zeros((n,n),dtype=np.float128)
    belief_y=np.zeros((n,n),dtype=np.float128)
    
    
    for t in range(max_iter):
        
        #m_x2y_new
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    sum_xi=0             #sum over all x_i
                    for xi in range(n):
#                         print(i,j,xi,yj,not((xi==j)!=(yj==i)))
                        if not((xi==j)!=(yj==i)): # if psi(x_i,y_j) is non zero
                            prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
                            for k in range(n):
                                if k!=j:
                                    prod_m_yk2xi*=m_y2x[k,i,xi]
                            sum_xi+=phi[i,xi]*prod_m_yk2xi  # the whole summation (over x_i)
                    m_x2y_new[i,j,yj]=sum_xi # message from x_i to y_j for a particular value of y_j
#                     print(i,j,yj,sum_xi)
        m_x2y_new=m_x2y_new/np.sum(m_x2y_new) #normalization
        
        #m_y2x_new
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    sum_yj=0            
                    for yj in range(n):        
                        if not((xi==j)!=(yj==i)): 
                            prod_m_xk2yj=1     
                            for k in range(n):
                                if k!=i:
                                    prod_m_xk2yj*=m_x2y[k,j,yj]
                            sum_yj+=phi[yj,j]*prod_m_xk2yj  
                    m_y2x_new[j,i,xi]=sum_yj         
        m_y2x_new=m_y2x_new/np.sum(m_y2x_new)#normalization
        
        
        #update using log rule
        for i in range(n):
            for j in range(n):
                for yj in range(n):
                    if m_x2y_new[i,j,yj] >0:
                        m_x2y[i,j,yj]=np.exp(np.log(m_x2y[i,j,yj]) + eps*(np.log(m_x2y_new[i,j,yj]) - np.log(m_x2y[i,j,yj])))
    #                   print("#",i,j,yj,(m_x2y[i,j,yj]),(m_x2y_new[i,j,yj]))
                    
        for j in range(n):
            for i in range(n):
                for xi in range(n):
                    if m_y2x_new[j,i,xi]>0:
                        m_y2x[j,i,xi]=np.exp(np.log(m_y2x[j,i,xi]) + eps*(np.log(m_y2x_new[j,i,xi]) - np.log(m_y2x[j,i,xi])))                   
                    
        m_x2y=m_x2y/np.sum(m_x2y) #normalization
        m_y2x=m_y2x/np.sum(m_y2x)#normalization

        
#         print(t, np.sum(np.abs(m_y2x-m_y2x_new)))

    #compute joint beliefs for each i,j and for each value of x_i and y_j
#     for i in range(n):
#         for j in range(n):
#             for xi in range(n):
#                 for yj in range(n):
#                     if not((xi==j)!=(yj==i)):
#                         prod_m_yk2xi=1     # prod_{k!=j}m_{y_k}(x_i)
#                         prod_m_xl2yj=1
#                         for k in range(n):
#                             if k!=j:
#                                 prod_m_yk2xi*=m_y2x[k,i,xi]
#                         for l in range(n):
#                             if l!=i:
#                                 prod_m_xl2yj*=m_x2y[l,j,yj]
#                         belief_xy[i,j,xi,yj]=phi[i,xi]*phi[yj,j]*prod_m_yk2xi*prod_m_xl2yj

                    
    
#     # normalize the joint beliefs                    
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #compute single node beliefs
    for i in range(n):
        for xi in range(n):
            prod_m_yk2xi=1
            for k in range(n):
                prod_m_yk2xi*=m_y2x[k,i,xi]
            belief_x[i,xi]=phi[i,xi]*prod_m_yk2xi
    
    for j in range(n):
        for yj in range(n):
            prod_m_xl2yj=1
            for l in range(n):
                prod_m_xl2yj*=m_x2y[l,j,yj]
            belief_y[j,yj]=phi[yj,j]*prod_m_xl2yj
            
    belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if not((xi==j)!=(yj==i)):
                        belief_xy[i,j,xi,yj]=belief_x[i,xi]*belief_y[j,yj]/(m_x2y[i,j,yj]*m_y2x[j,i,xi])
                    else:
                        belief_xy[i,j,xi,yj]=0
    
    for i in range(n):
        for j in range(n):
            belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])
    
    #normalize single node beliefs
    for i in range(n):
        belief_x[i,:]=belief_x[i,:]/np.sum(belief_x[i,:])
            
    for j in range(n):
        belief_y[j,:]=belief_y[j,:]/np.sum(belief_y[j,:])
        
#     belief_xy=belief_xy/n
#     belief_x=belief_x/(n)
#     belief_y=belief_y/(n)
        
    return m_x2y,m_y2x,belief_xy,belief_x,belief_y,(np.sum(np.abs(m_y2x-m_y2x_new))<1e-10),np.sum(np.abs(m_y2x-m_y2x_new))

In [7]:
def bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x):
        
    t1=t2=t3=t4=t5=t6=0
    n=P.shape[0]
#     belief_x=np.zeros((n,n),dtype=float)
#     belief_y=np.zeros((n,n),dtype=float)
    phi=np.sqrt(P)
    
#     belief_xy=np.zeros((n,n,n,n),dtype=np.float128)
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=np.outer(belief_x[i,:],belief_y[j,:])/(m_x2y[i,j]*m_y2x[j,i])
#     for i in range(n):
#         for j in range(n):
#             belief_xy[i,j,:,:]=belief_xy[i,j,:,:]/np.sum(belief_xy[i,j,:,:])

    
#     for i in range(n):
#         belief_x[i,:]=np.sum(belief_xy[i,0],axis=1)
#         belief_y[i,:]=np.sum(belief_xy[0,i],axis=0)
        
    for i in range(n):
        for j in range(n):
            for xi in range(n):
                for yj in range(n):
                    if belief_xy[i,j,xi,yj]!=0 and not((xi==j)!=(yj==i)):
                        t1+=belief_xy[i,j,xi,yj]*np.log(phi[i,xi]*phi[yj,j])
                        t2+=belief_xy[i,j,xi,yj]*np.log(belief_xy[i,j,xi,yj])
    
    for i in range(n):
        for xi in range(n):
            if belief_x[i,xi]!=0:
                t3+=belief_x[i,xi]*np.log(belief_x[i,xi])
                t5+=belief_x[i,xi]*np.log(phi[i,xi])
                                          
    for j in range(n):
        for yj in range(n):
            if belief_y[j,yj]!=0:
                t4+=belief_y[j,yj]*np.log(belief_y[j,yj])
                t6+=belief_y[j,yj]*np.log(phi[yj,j])
    
#     print('t1', -t1)    
#     print('t2', t2)
#     print('t3', -(n-1)*t3)
#     print('t4', -(n-1)*t4)
#     print('t5', (n-1)*t5)
#     print('t6', (n-1)*t6)
    energy=-t1+t2-(n-1)*t3-(n-1)*t4+(n-1)*t5+(n-1)*t6
#     print(energy)
    return np.exp(-energy)

In [8]:
def f_bp(P):
    
    def f_bp_p(beta):
        n=P.shape[1]
        z=0
#         print(beta.sum())
        beta=beta.reshape(n,n)
#         print(beta)
        for i in range(n):
            for j in range(n):
                
#                 if beta[i,j]<0:
#                     print(beta[i,j])
                if beta[i,j]!=0 and beta[i,j]!=1 and beta[i,j]>0 and P[i,j]>0: 
                    z+=beta[i,j]*np.log(beta[i,j]/P[i,j])-(1-beta[i,j])*np.log(1-beta[i,j])
#                 print((beta[i,j]/P[i,j]))
        return z
    return f_bp_p

# Experiments

#### Function to generate random matrices for given $\kappa$

In [9]:
def random_sparse_matrix(rng,n,kappa):
    mask=rng.binomial(1,kappa,(n,n))
    # mat=rng.exponential(size=(n,n))
    mat=rng.uniform(high=2.0,size=(n,n))
    # mat=rng.gamma(2,size=(n,n))
    return np.multiply(mask,mat)


#### Compute permanent for each n with sparsity c/n

In [ ]:
rng=np.random.default_rng(101)

for c in np.linspace(1,3,num=9)[0:1]:
    eta_avg_arr=np.zeros(10)
    samples=100
    # print('c: ',c)
    for n in range(3,9):
        eta_sum=0
        k=c/n
        # print('n: ',n)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)
            
            #preprocessing step, vatedka et al
            # r,c=linear_sum_assignment(P,maximize=True)
            # P=P[:,c]
            
            # print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta=np.log2(perm/perm_bethe)/P.shape[0]
            eta_sum+=eta
            print(f'c: {c}, n: {n}, sample: {t}')
            print("Perm: \t",perm)
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Conv Value: ", conv_value)
            print("Eta: \t",eta)
            print('\n\n\n')
            np.savez(drive_path+f'linear_exp/exp_linear_c_{c}_n_{n}_sample_{t}',
                     perm=perm,perm_bethe=perm_bethe,eta=eta)
        eta_avg_arr[n]=eta_sum/samples
        # print('eta_avg_arr ',eta_avg_arr)
    np.save(drive_path+'linear_exp/eta_exp_linear_c_'+str(c),eta_avg_arr)

c: 1.0, n: 3, sample: 0
Perm: 	 2.148131956745184418
Bethe: 	 2.1481319567451838699
Bounds: 	 True
Conv Value:  0.06587684538804289232
Eta: 	 1.2273544905566919386e-16




c: 1.0, n: 3, sample: 1
Perm: 	 2.1454879101399622174
Bethe: 	 2.1033824640184478594
Bounds: 	 True
Conv Value:  0.06233859768339594999
Eta: 	 0.009531522743890984625




c: 1.0, n: 3, sample: 2
Perm: 	 0.1640427694934392974
Bethe: 	 0.16127667361023409445
Bounds: 	 True
Conv Value:  0.06496833226636597597
Eta: 	 0.008178072498804265231




c: 1.0, n: 3, sample: 3
Perm: 	 0.51683544585376065277
Bethe: 	 0.44938467945689088953
Bounds: 	 True
Conv Value:  0.057141566857850523937
Eta: 	 0.06725135880367441158




c: 1.0, n: 3, sample: 4
Perm: 	 0.9910154994881999048
Bethe: 	 0.6266073002993145411
Bounds: 	 True
Conv Value:  0.061115983115211085508
Eta: 	 0.22044868113212882417




c: 1.0, n: 3, sample: 5
Perm: 	 0.2889446893325718854
Bethe: 	 0.28442737326493885812
Bounds: 	 True
Conv Value:  0.064831739291926131086
Eta

#### Compute permanent for each $\kappa$ and n


In [ ]:
rng=np.random.default_rng(2)
for n in range(3,10):
    samples=10
    eta_avg_arr=np.zeros(10)
    idx=0
    for k in np.linspace(0,1,num=11)[1:10]:
        eta_sum=0
        print('k: ',k)
        for t in range(0,samples):
            P=random_sparse_matrix(rng,n,k)
            perm=permanent(P)
            
            while (np.isclose(perm,0)):
                P=random_sparse_matrix(rng,n,k)
                perm=permanent(P)

            # print(P)
            m_x2y,m_y2x,belief_xy,belief_x,belief_y,conv,conv_value=belief_prop(P,300)
            perm_bethe=bfe(P,belief_xy,belief_x,belief_y,m_x2y,m_y2x)
            ub=perm_bethe*np.sqrt(2)**P.shape[0]
            eta_sum+=np.log(perm/perm_bethe)/P.shape[0]

            print("Perm: \t",perm)
            print("Bethe: \t" ,perm_bethe)
            print("Bounds: \t",perm_bethe<perm<ub)
            print("Convergence: \t",conv)
            if not conv:
                print("Conv Value: ", conv_value)
            print("Eta: \t",np.log(perm/perm_bethe)/P.shape[0])
            print('\n\n\n')
        eta_avg_arr[idx]=eta_sum/samples
        idx+=1
    np.save('eta_avg_arr_gamma_n_'+str(n),eta_avg_arr)

#### Print table

In [ ]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
path=drive_path+'eta_exp_linear_c_'
data1=np.load(path+'1.0.npy')
data2=np.load(path+'1.25.npy')
data3=np.load(path+'1.5.npy')
data4=np.load(path+'1.75.npy')
data5=np.load(path+'2.0.npy')
print(data1[3:]*np.log2(np.e))
print(data2[3:]*np.log2(np.e))
print(data3[3:]*np.log2(np.e))
print(data4[3:]*np.log2(np.e))
print(data5[3:]*np.log2(np.e))

# path=drive_path+'gamma/eta_avg_arr_gamma_n_'
# data1=np.load(path+'3.npy')
# data2=np.load(path+'4.npy')
# data3=np.load(path+'5.npy')
# data4=np.load(path+'6.npy')
# data5=np.load(path+'7.npy')
# data6=np.load(path+'8.npy')
# data7=np.load(path+'9.npy')

# print(data1*np.log2(np.e))
# print(data2*np.log2(np.e))
# print(data3*np.log2(np.e))
# print(data4*np.log2(np.e))
# print(data5*np.log2(np.e))
# print(data6*np.log2(np.e))
# print(data7*np.log2(np.e))

[0.0496 0.0211 0.0197 0.0216 0.0175 0.0172 0.0000]
[0.0539 0.0387 0.0437 0.0329 0.0359 0.0266 0.0000]
[0.0843 0.0472 0.0599 0.0426 0.0430 0.0329 0.0000]
[0.0662 0.0266 0.0654 0.0394 0.0275 0.0556 0.0000]
[0.0666 0.1262 0.0942 0.0558 0.0929 0.0617 0.0000]


In [10]:
a=np.load('/content/drive/My Drive/Colab Notebooks/data_IISc experiment2/linear_uniform/eta_uniform_linear_c_1.0.npy')

array([0.        , 0.        , 0.        , 0.02569969, 0.03708991,
       0.03422888, 0.02173701, 0.02105168, 0.02188653, 0.        ])